# __Heaps__

A heap is a tree-based data structure that satisfies the __heap property__:

- In a __min-heap__, each parent node is no __greater__ than its children.

	&emsp; $P <= C \quad \forall \ \text{children} \ C \ \text{of} \ P$

- In a __max-heap__, each parent node is no __less than__ its children.

	&emsp; $P >= C \quad \forall \ \text{children} \ C \ \text{of} \ P$

> Where $P$ is some parent node in a heap.

Therefore, heaps are only partially sorted but they offer $O(1)$ time access to the min/max element, which is always on top.

Heaps are typically implemented as binary trees, called __binary heaps__, which are:

- __Complete binary trees__ 

	- Each node has up to two children

	- All levels are filled except possibly the last, which is filled left-to-right

- Stored efficiently in arrays, where:

	- Index `0` is the root (min or max element)

	- Index `i` has its children at `2i +1` and `2i + 2`

	- And its parent at `(i - 1) // 2`

### Understanding Array Indexing in Binary Heaps

```mermaid
graph TD
    A[10]
    B[20]
    C[15]
    D[30]
    E[40]
    F[50]
    G[60]

    A --> B
    A --> C
    B --> D
    B --> E
    C --> F
    C --> G

```

The nodes are laid into an array in __level-order traversal__ (like a BFS).

__Corresponding Array__:

`[10, 20, 15, 30, 40, 50, 60]`

Binary heaps are read from top-down, left-to-right.

So, `10` is the root (`i=0`), 
- `20` is `i=1`
- `15` is `i=2`

- &ensp;&ensp;&ensp; &hellip;
- `60` is `i=6`

#### Why `2i + 1` and `2i + 2` Work

- A node `cur` at index `i` has exactly `i` nodes before it in the tree

- Since the tree is complete, all the nodes before `cur` have 2 children

- Each node `prev`, occurring before `cur` in the tree, will have its children shown before `cur`'s children

	- Therefore, `2i` children will be shown before `cur`'s children

- Every node except for the root are children

	- Therefore,  `2i + 1` nodes occur before `cur`'s first child

- Recap:
	- There are `i` nodes before node `cur`

	- Each having exactly 2 children

	- This accounts for all nodes before `cur`, except for the root (which is not a child)

	- We add 1 for the root, then `2i + 1` nodes occur before `cur`'s first child

	- Using 0-indexing, the $(2i + 1)^{st}$ node has `2i + 1` elements before it

	- Therefore, `cur` at `i` has a child at `2i + 1` and its next child is adjacent `2i + 2`

### Why `(i - 1) // 2` Works

Notice that the first child of a node is always at an odd index,  
and the second child is always at an even index (one greater than the first child).

We have:

$$\begin{align*} 
C_1 &= 2P + 1 \\
C_2 = C_1 + 1 &= 2P + 2
\end{align*}
$$

Inverse functions:

$$ P = \frac{C_1 - 1}{2} $$

$$ P = \frac{C_2 - 2}{2} $$

We define a function to work in both cases (for either child):

$$ P = \left\lfloor \frac{C - 1}{2} \right\rfloor $$

- If $C$ is odd (first child):

	$\displaystyle P = \left\lfloor \frac{2k + 1 - 1}{2} \right\rfloor = k $

- If $C$ is even (second child):

	$\displaystyle P = \left\lfloor \frac{2k + 2 - 1}{2} \right\rfloor = \left\lfloor k + \frac{1}{2} \right\rfloor = k $

### 👍 __Core Heap Operations__

| Operation | Description | Time Complexity |
| --------- | ----------- | ---------------:|
| `peek()` | Return the top element (min or max) | $O(1)$ |
| `pop()` | Remove and return the top element | $O({log}_2 \ n)$ |
| `push(x)` | Insert a new item into the heap | $O({log}_2 \ n)$ |
| `replace(x)` | Insert a new item into the heap | $O({log}_2 \ n)$ |


### 🛠️ __Creation__

| Operation | Description | Time Complexity |
| --------- | ----------- | ---------------:|
| `create_heap()` | Creates an empty heap structure | $O(1)$ |
| `heapify(array)` | Builds a heap from a given array of elements | $O(n)$ |
| `merge(h1, h2)` | Combines two heaps, preserving the original ones | $O(n + m)$ |
| `meld(h1, h2)` | Combines two heaps and destroys the originals | $O(n + m)$ |

### 🔍 __Inspection__

| Operation | Description | Time Complexity |
| --------- | ----------- | ---------------:|
| `size()` | Returns the number of nodes | $O(1)$ |
| `is_empty()` | Checks if heap is empty | $O(1)$ |
| `contains(x)` | Linear scan for element | $O(n)$ |


### ⚙️ __Internal__

| Operation | Description | Time Complexity |
| --------- | ----------- | ---------------:|
| `update_key(i, new_key)` | Change the key-value associated with a node | $O({log}_2 \ n)$ |
| `delete(i)` | Remove an arbitrary node from the tree | $O({log}_2 \ n)$ |
| `sift_up(i)` | Move a node up the tree until the heap property is restored | $O({log}_2 \ n)$ |
| `sift_down(i)` | Move a node down until the heap property is restored | $O({log}_2 \ n)$ |
| `swap(i, j)` | Swaps two nodes in-place | $O(1)$ |

# __Let's Build a Heap in Python__

In [1]:
from abc import ABC, abstractmethod

class Heap(ABC):
	@abstractmethod # Cannot be instantiated directly, must be subclassed and overridden
	def __init__(self, cmp):
		self.heap = []
		self.cmp = cmp
		''' Comparison function: (a, b) -> True if node `b` should move above `a` in the heap
			- lt for min-heap
			- gt for max-heap
		'''

	@classmethod
	def heapify(cls, array):
		''' Turn a list into a Heap object (the array is referenced) '''
		new_heap = cls()
		new_heap.heap = array
		new_heap._heapify()
		return new_heap

	# Core
	def push(self, node):
		self.heap.append(node)
		self._sift_up(len(self.heap) - 1)

	def pop(self):
		if not self.heap:
			raise IndexError('Heap is empty')
		top = self.heap[0]
		last = self.heap.pop()
		if self.heap:
			self.heap[0] = last
			self._sift_down(0)
		return top
	
	def peek(self):
		if not self.heap:
			raise IndexError('Heap is empty')
		return self.heap[0]
	
	def replace(self, node):
		''' Replaces the top element and restores the heap property. '''
		if not self.heap:
			self.heap.append(node)
		else:
			self.heap[0] = node
			self._sift_down(0)

	def merge(self, other):
		''' Return merged heap without destroying input heaps. '''
		return type(self).heapify(self.heap + other.heap)

	def meld(self, other):
		''' Merge `other` into current heap, destroying `other`. '''
		self.heap.extend(other.heap)
		self._heapify()
		del other

	def is_empty(self):
		return not self.heap
	
	def update_node(self, i, new_node):
		old_node = self.heap[i]
		self.heap[i] = new_node
		if self.cmp(new_node, old_node):
			self._sift_up(i)
		else:
			self._sift_down(i)

	def delete(self, i):
		''' Removes the node at index `i`, and maintains the heap property '''
		if i >= len(self.heap):
			raise IndexError('Index out of range')
		
		last = self.heap.pop()
		if i < len(self.heap): # If not deleting last node
			self.heap[i] = last
			if i > 0 and self.cmp(self.heap[i], self.heap[(i - 1) // 2]):
				self._sift_up(i)
			else:
				self._sift_down(i)

	# Internal
	def _heapify(self):
		''' Restore heap property for an unsorted array. '''
		for i in reversed(range(len(self.heap) // 2)):
			self._sift_down(i)

	def _sift_up(self, i):
		while i > 0:
			parent = (i - 1) // 2
			if self.cmp(self.heap[i], self.heap[parent]): # Swap
				self.heap[i], self.heap[parent] = self.heap[parent], self.heap[i]
				i = parent
			else:
				break # Heap property satisfied

	def _sift_down(self, i):
		n = len(self.heap)
		while True:
			# Children
			left = 2 * i + 1
			right = 2 * i + 2
			# Find the min/max key from parent i and its 2 children
			extreme = i
			if left < n and self.cmp(self.heap[left], self.heap[extreme]):
				extreme = left
			if right < n and self.cmp(self.heap[right], self.heap[extreme]):
				extreme = right
			if extreme != i: # Swap
				self.heap[i], self.heap[extreme] = self.heap[extreme], self.heap[i]
				i = extreme
			else:
				break
	
	def __len__(self):
		return len(self.heap)
	
	def __contains__(self, val):
		return val in self.heap
	
	def __str__(self):
		return str(self.heap)
	
	def __repr__(self):
		return f'{self.__class__.__name__}(heap={self.heap})'
	

class MinHeap(Heap):
	def __init__(self):
		super().__init__(cmp=lambda a, b: a < b)

class MaxHeap(Heap):
	def __init__(self):
		super().__init__(cmp=lambda a, b: a > b)

In [2]:
# Provide keys to any data
class Node:
	def __init__(self, key, data):
		self.key = key # For comparisons
		self.data = data
	
	def __lt__(self, other):
		return self.key < other.key
	
	def __le__(self, other):
		return self.key <= other.key
	
	def __eq__(self, other):
		return self.key == other.key
	
	def __str__(self):
		return f'{self.key}: {self.data}'
	
	def __repr__(self):
		return f'Node(key={self.key}, data={self.data})'

## Example Problem

#### Find the `k` largest elements in an array:

- Push the first `k` elements into a min-heap

- For each remaining element, if it's greater than the heap's top (min) element, replace min with the new element

- At the end, the heap contains the `k` largest elements

In [3]:
# Brute force approach first (no heap)
def k_largest_bruteforce(nums, k):
	nums_sorted = sorted(nums, reverse=True) # Copies and sorts array
	return nums_sorted[:k]

#### Brute Force

Time: $O(n \cdot log \ n)$, &ensp; Where $n$ is `len(nums)`

Space: $O(n)$, &ensp; Copies new array of size $n$

In [4]:
# Min-Heap Solution
def k_largest_heap(nums, k):
	min_heap = MinHeap()
	for n in nums:
		if len(min_heap) < k: # Add first k elements to min-heap
			min_heap.push(n)
		else: # Replace greater elements with heap top (min)
			if n > min_heap.peek():
				min_heap.replace(n)
	return min_heap

#### Min-Heap

Time: $O(n \cdot log \ k)$, &ensp; Heap replace with a heap of size `k`, at most `n` times

Space: $O(k)$, &ensp; Heap is always size `k`

In [5]:
import time
import numpy as np

def test_k_largest(nums, k):
	start = time.perf_counter()
	brute_list = k_largest_bruteforce(nums, k)
	brute_time = time.perf_counter() - start

	start = time.perf_counter()
	heap = k_largest_heap(nums, k)
	heap_time = time.perf_counter() - start

	heap_list = sorted(heap.heap, reverse=True)

	# Ensure the solutions are the same
	assert brute_list == heap_list, 'Results differ!'

	print(f'Brute force: {brute_time:.6f}s')
	print(brute_list)
	print()

	print(f'Min-Heap: {heap_time:.6f}s')
	print(heap_list)
	print()

# Array of 10 million integers from 1..10 million
array_np = np.random.randint(1, 10**7 + 1, size=10**7)
array = array_np.tolist()

In [6]:
test_k_largest(array, k=10)

Brute force: 4.543292s
[9999999, 9999998, 9999997, 9999996, 9999996, 9999995, 9999994, 9999993, 9999991, 9999991]

Min-Heap: 3.625022s
[9999999, 9999998, 9999997, 9999996, 9999996, 9999995, 9999994, 9999993, 9999991, 9999991]

